In [1]:
from transformers import TFBertModel
from transformers import AutoTokenizer, BertModel
import os
os.environ['CURL_CA_BUNDLE'] = ''
import torch
from dotenv import load_dotenv
from langchain_community.graphs import Neo4jGraph

Load BERT Model (later we can change to other model)

In [6]:
# Choose the pretrained model
model_checkpoint = 'bert-base-multilingual-cased' #'bert-base-multilingual-cased'   #'google/bert_uncased_L-2_H-128_A-2' (Tiny BERT)
# Max number of tokens in the sentence
max_length= 512 #512 #128

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, model_max_length= max_length)

# Load the model
#bert_model = TFBertModel.from_pretrained(model_checkpoint, from_pt=True)
bert_model = BertModel.from_pretrained(model_checkpoint)

c:\Users\facordei\OneDrive - Capgemini\Documents\GitHub\Indigenous-Slavery-KG\.venv-IndigSlave\Lib\site-packages\urllib3\connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\facordei\OneDrive - Capgemini\Documents\GitHub\Indigenous-Slavery-KG\.venv-IndigSlave\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
c:\Users\facordei\OneDrive - Capgemini\Documents\GitHub\Indigenous-Slavery-KG\.venv-IndigSlave\Lib\site-packages\urllib3\connectionpool.py:1064: Insecure

In [7]:
# The function receives a sencence and returns the embedding (1D numpy array)
def get_embeddings(text):
  input_ids = torch.tensor(tokenizer.encode(text, add_special_tokens=True, truncation=True)).unsqueeze(0)  # Batch size 1
  outputs = bert_model(input_ids)
  last_hidden_states = outputs[0]
  pooler_output = torch.mean(last_hidden_states, dim=1) 
  return(pooler_output.detach().numpy()[0].tolist())

Connecting to Neo4j

In [2]:
load_dotenv()

# Neo4j variables
NEO4J_URL = os.getenv("NEO4J_URL")
NEO4J_USERNAME =os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

#Connecting to the graph
graph = Neo4jGraph(
    url=NEO4J_URL,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD
)

In [9]:
#Quering the graph unsing Cypher
result = graph.query("""
MATCH (thesis:Thesis)
RETURN thesis.uri, thesis.title, thesis.abstract
""")

In [10]:
dic_embeddings = {}
for thesis in result:
    try:
        dic_embeddings[thesis['thesis.uri']] = get_embeddings(str(thesis['thesis.title']) + '\n   \n' + str(thesis['thesis.abstract']))
    except:
        print(thesis['thesis.uri'])
        print(thesis['thesis.title'])
        print(thesis['thesis.abstract'])
        print('--------------')

In [3]:
import json

#with open('data/embeddings.json', 'w') as fp:
#    json.dump(dic_embeddings, fp)

with open('data/embeddings.json', 'r') as fp:
    dic_embeddings = json.load(fp)

In [32]:
n = 0
small_dic = {}
for i in dic_embeddings:
    n = n + 1
    small_dic[i]=dic_embeddings[i]
    if n == 10:
        break

with open('data/small_embeddings.json', 'w') as fp:
    json.dump(small_dic, fp)

In [33]:
small_dic

{'tag:stardog:api:_e_teko_e_arandu_e_producao_de_subjetividades_e_educacao_superior_e_educacoes_outras_modos_de_vida_criados_e_afirmados_por_kaiowas_e_guaranis': [-0.036806486546993256,
  0.053812891244888306,
  0.4343889653682709,
  0.07008563727140427,
  0.06726423650979996,
  -0.0707978904247284,
  -0.1256973147392273,
  0.12517786026000977,
  -0.15045271813869476,
  -0.0530383475124836,
  -0.14858010411262512,
  0.36490532755851746,
  0.21294069290161133,
  0.293968141078949,
  0.08269951492547989,
  -0.24902383983135223,
  0.5886573195457458,
  -0.3843301236629486,
  -0.5035964846611023,
  -0.09140083193778992,
  -0.20768438279628754,
  0.14955902099609375,
  -0.16800528764724731,
  -0.12197914719581604,
  -0.01872563175857067,
  0.49669182300567627,
  -0.49017098546028137,
  -0.07764355838298798,
  -0.5541002750396729,
  -0.4451574385166168,
  0.019914740696549416,
  0.2981143593788147,
  -0.08936356753110886,
  0.43163129687309265,
  -0.08834492415189743,
  -0.1539146602153778,


In [30]:
ini = 'CALL apoc.import.json("'
dic = str(small_dic)
fim = '" YELD'
print(ini + dic + fim)
result = graph.query(ini + dic + fim)

result

CALL apoc.import.json("{'tag:stardog:api:_e_teko_e_arandu_e_producao_de_subjetividades_e_educacao_superior_e_educacoes_outras_modos_de_vida_criados_e_afirmados_por_kaiowas_e_guaranis': [-0.036806486546993256, 0.053812891244888306, 0.4343889653682709, 0.07008563727140427, 0.06726423650979996, -0.0707978904247284, -0.1256973147392273, 0.12517786026000977, -0.15045271813869476, -0.0530383475124836, -0.14858010411262512, 0.36490532755851746, 0.21294069290161133, 0.293968141078949, 0.08269951492547989, -0.24902383983135223, 0.5886573195457458, -0.3843301236629486, -0.5035964846611023, -0.09140083193778992, -0.20768438279628754, 0.14955902099609375, -0.16800528764724731, -0.12197914719581604, -0.01872563175857067, 0.49669182300567627, -0.49017098546028137, -0.07764355838298798, -0.5541002750396729, -0.4451574385166168, 0.019914740696549416, 0.2981143593788147, -0.08936356753110886, 0.43163129687309265, -0.08834492415189743, -0.1539146602153778, 0.0964585542678833, -0.12782658636569977, 0.520

CypherSyntaxError: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'YELD': expected
  "!="
  "%"
  ")"
  "*"
  "+"
  ","
  "-"
  "/"
  "::"
  "<"
  "<="
  "<>"
  "="
  "=~"
  ">"
  ">="
  "AND"
  "CONTAINS"
  "ENDS"
  "IN"
  "IS"
  "OR"
  "STARTS"
  "XOR"
  "^"
  "||" (line 1, column 16429 (offset: 16428))
"CALL apoc.import.json("{'tag:stardog:api:_e_teko_e_arandu_e_producao_de_subjetividades_e_educacao_superior_e_educacoes_outras_modos_de_vida_criados_e_afirmados_por_kaiowas_e_guaranis': [-0.036806486546993256, 0.053812891244888306, 0.4343889653682709, 0.07008563727140427, 0.06726423650979996, -0.0707978904247284, -0.1256973147392273, 0.12517786026000977, -0.15045271813869476, -0.0530383475124836, -0.14858010411262512, 0.36490532755851746, 0.21294069290161133, 0.293968141078949, 0.08269951492547989, -0.24902383983135223, 0.5886573195457458, -0.3843301236629486, -0.5035964846611023, -0.09140083193778992, -0.20768438279628754, 0.149559020

In [8]:
str(small_dic)

"{'tag:stardog:api:_e_teko_e_arandu_e_producao_de_subjetividades_e_educacao_superior_e_educacoes_outras_modos_de_vida_criados_e_afirmados_por_kaiowas_e_guaranis': [-0.036806486546993256, 0.053812891244888306, 0.4343889653682709, 0.07008563727140427, 0.06726423650979996, -0.0707978904247284, -0.1256973147392273, 0.12517786026000977, -0.15045271813869476, -0.0530383475124836, -0.14858010411262512, 0.36490532755851746, 0.21294069290161133, 0.293968141078949, 0.08269951492547989, -0.24902383983135223, 0.5886573195457458, -0.3843301236629486, -0.5035964846611023, -0.09140083193778992, -0.20768438279628754, 0.14955902099609375, -0.16800528764724731, -0.12197914719581604, -0.01872563175857067, 0.49669182300567627, -0.49017098546028137, -0.07764355838298798, -0.5541002750396729, -0.4451574385166168, 0.019914740696549416, 0.2981143593788147, -0.08936356753110886, 0.43163129687309265, -0.08834492415189743, -0.1539146602153778, 0.0964585542678833, -0.12782658636569977, 0.5204546451568604, 0.35697

In [ ]:
result = graph.query("""
    WITH apoc.util.compress('""" + small_dic + """
    CALL apoc.import.json(jsonCompressed, {compression: 'DEFLATE'})
    CALL apoc.load.json("file:///Users/facordei/OneDrive - Capgemini/Documents/GitHub/Indigenous-Slavery-KG/data/embeddings.json")
    YIELD value
    """)

result

WITH apoc.util.compress('{"type":"node","id":"2","labels":["User"],"properties":{"age":12}}', {compression: 'DEFLATE'}) AS jsonCompressed
CALL apoc.import.json(jsonCompressed, {compression: 'DEFLATE'})
YIELD source, format, nodes, relationships, properties
RETURN source, format, nodes, relationships, properties

In [ ]:
#Load the embeddings in the Noe4j database
n = 0
for uri in dic_embeddings:

    result = graph.query("""
    MATCH (thesis:Thesis) 
    Where (thesis.uri = '""" + uri + """')
    CALL db.create.setNodeVectorProperty(thesis, 'embedding', """ + str(dic_embeddings[uri]) + """)
    """)

    n = n + 1
    if n % 50 == 0:
        print (uri)
    

#MATCH (thesis:Thesis) 
#Where (thesis.uri = "tag:stardog:api:_nos_aqui_e_o_espaco_dos_sem_vez__quilombolas_e_educacao_em_poconemt")
#CALL db.create.setNodeVectorProperty(thesis, 'embedding', [0,1,2,3,4])
#RETURN thesis.uri, thesis.title, thesis.abstract

In [6]:
result = graph.query("""
    CALL apoc.load.json("file:///Users/facordei/OneDrive - Capgemini/Documents/GitHub/Indigenous-Slavery-KG/data/embeddings.json")
    YIELD value
    """)

result

ClientError: {code: Neo.ClientError.Procedure.ProcedureCallFailed} {message: Failed to invoke procedure `apoc.load.json`: Caused by: java.lang.RuntimeException: Import from files not enabled, please set apoc.import.file.enabled=true in your apoc.conf}

In [148]:
import os
from getpass import getpass

import numpy as np
import tensorflow as tf
from transformers import TFBertModel
from transformers import AutoTokenizer

from langchain_community.graphs import Neo4jGraph

from sklearn.metrics.pairwise import cosine_similarity

from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt, mpld3

### Load BERT model, NEO4J database and generating embeddings

In [2]:
# Choose the pretrained model
model_checkpoint = 'bert-base-multilingual-cased' #'bert-base-multilingual-cased'   #'google/bert_uncased_L-2_H-128_A-2' (Tiny BERT)
# Max number of tokens in the sentence
max_length= 512 #512 #128

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, model_max_length= max_length)

In [3]:
# Load the model
bert_model = TFBertModel.from_pretrained(model_checkpoint, from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already

In [4]:
# The function receives a sencence and returns the embedding (1D numpy array)
def get_embeddings(text):
  inputs = tokenizer(text, truncation=True, return_tensors="tf")
  outputs = bert_model(inputs)

  last_hidden_states = outputs.last_hidden_state
  pooler_output = outputs.pooler_output
  return(pooler_output.numpy()[0].tolist())

In [5]:
# Example
text = "Hello, my dog is cute"
get_embeddings(text)

[0.3043421804904938,
 0.0655800998210907,
 0.2867922782897949,
 -0.18544849753379822,
 -0.13707706332206726,
 0.5639879703521729,
 0.22897112369537354,
 0.20353536307811737,
 -0.47550225257873535,
 0.43214404582977295,
 -0.10113587975502014,
 -0.2794475257396698,
 -0.2155974805355072,
 -0.1278197020292282,
 0.21039064228534698,
 -0.21768882870674133,
 0.709071934223175,
 -0.009180756285786629,
 0.19599683582782745,
 -0.4344290494918823,
 -0.9999848008155823,
 -0.120848149061203,
 -0.34574177861213684,
 -0.21058939397335052,
 -0.3446536064147949,
 0.1665513664484024,
 -0.2625880837440491,
 0.07631348818540573,
 0.19793269038200378,
 -0.19297268986701965,
 0.1113186627626419,
 -0.999985933303833,
 0.5718837380409241,
 0.7091876864433289,
 0.24170824885368347,
 -0.09670403599739075,
 0.22048485279083252,
 0.27047964930534363,
 0.2205585539340973,
 -0.3865535855293274,
 -0.26536279916763306,
 -0.09083323925733566,
 -0.1782226413488388,
 0.24492590129375458,
 -0.18337216973304749,
 -0.24588

Connecting Neo4j database

In [6]:
# Neo4j variables
NEO4J_URL = getpass("NEO4J_URL")
NEO4J_USERNAME = getpass("NEO4J_USERNAME")
NEO4J_PASSWORD = getpass("NEO4J_PASSWORD")

NEO4J_URL ········
NEO4J_USERNAME ········
NEO4J_PASSWORD ········


In [313]:
#Connecting to the graph
graph = Neo4jGraph(
    url=NEO4J_URL,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD
)

In [314]:
#Quering the graph using Cypher
result_LRA = graph.query("""
MATCH (n:LRA) RETURN ID(n), n.name, n.data_description;
""")

In [9]:
# Loop to creating the embeddings for LRA
embeddings_LRA = []

i = 0
for text in result_LRA:

    if text['n.name'] == None:
        text['n.name'] = " "
    if text['n.data_description'] == None:
        text['n.data_description'] = " "
   
    name_embeddings = get_embeddings(text['n.name'])
    description_embeddings = get_embeddings(text['n.data_description'])
    nameANDdescription_embeddings = get_embeddings(text['n.name']+ ' - ' +text['n.data_description'])
   
    embeddings_LRA.append({'ID': text['ID(n)'],
                           'name': text['n.name'],
                           'descriptions': text['n.data_description'],
                           'name_embeddings': name_embeddings,  
                           'description_embeddings': description_embeddings,
                           'nameANDdescription_embeddings': nameANDdescription_embeddings})

    graph.query("MATCH (n:LRA) WHERE ID(n) = " + str(text['ID(n)']) + " SET n.name_embeddings = " + str(name_embeddings) + " SET n.description_embeddings = " + str(description_embeddings) + " SET n.nameANDdescription_embeddings = " + str(nameANDdescription_embeddings) + ";")


In [23]:
#Quering the graph using Cypher
result_BII = graph.query("""
MATCH (n:BII) RETURN ID(n), n.name, n.information_asset_details;
""")

In [25]:
# Loop to creating the embeddings for BII
embeddings_BII = []

i = 0
for text in result_BII:

    if text['n.name'] == None:
        text['n.name'] = " "
    if text['n.information_asset_details'] == None:
        text['n.information_asset_details'] = " "
   
    name_embeddings = get_embeddings(text['n.name'])
    description_embeddings = get_embeddings(text['n.information_asset_details'])
    nameANDdescription_embeddings = get_embeddings(text['n.name']+ ' - ' +text['n.information_asset_details'])
   
    embeddings_BII.append({'ID': text['ID(n)'],
                           'name': text['n.name'],
                           'descriptions': text['n.information_asset_details'],
                           'name_embeddings': name_embeddings,  
                           'description_embeddings': description_embeddings,
                           'nameANDdescription_embeddings': nameANDdescription_embeddings})

    graph.query("MATCH (n:BII) WHERE ID(n) = " + str(text['ID(n)']) + " SET n.name_embeddings = " + str(name_embeddings) + " SET n.description_embeddings = " + str(description_embeddings) + " SET n.nameANDdescription_embeddings = " + str(nameANDdescription_embeddings) + ";")

In [294]:
# Lists with name, descriptions and Embeddings

emb_LRA = []
name_LRA = []
desc_LRA = []
ID_LRA = []

for data in embeddings_LRA:
    name_LRA.append(data['name'])
    desc_LRA.append(data['descriptions'])
    emb_LRA.append(data['nameANDdescription_embeddings'])
    ID_LRA.append(data['ID'])


emb_BII = []
name_BII = []
desc_BII = []
ID_BII = []

for data in embeddings_BII:
    name_BII.append(data['name'])
    desc_BII.append(data['descriptions'])
    emb_BII.append(data['nameANDdescription_embeddings'])
    ID_BII.append(data['ID'])

### LRA x BII similarity

In [27]:
# Calculating a matrix with cosine similarity between LRA x BII
LRA_BII_cos = cosine_similarity(np.array(emb_LRA), np.array(emb_BII))

In [28]:
# Printing most similar BII data for each LRA data above the threshold.

th = 0.995

for n in range (len(LRA_BII_cos)):

    most_sim = np.argmax(LRA_BII_cos[n])
    
    if LRA_BII_cos[n][most_sim] > th:
            
        print ("LRA data: ", name_LRA[n])
        print ("LRA description: ", desc_LRA[n])
        print (" ")
        
        print("Most similar BII data: ", name_BII[most_sim])
        print("Most similar BII description: ", desc_BII[most_sim])
        print ("cosine similarity: ", LRA_BII_cos[n][most_sim])
        print ("------------")

LRA data:  Real-time well data
LRA description:  Real-Time drilling and logging data - Incl.cement log data. SiteCom + ROP (Rate of Penetration) + WOB (Weight on bit)+ TQ (Torque) + Hookload + MWD (Measurements while drilling) + LWD (Logging while drilling) + WITSML (Well Site Information Transfer Standard Markup Language)
 
Most similar BII data:  Real-Time log data
Most similar BII description:  Real-Time drilling and logging data - Incl.cement log data. SiteCom + ROP (Rate of Penetration) + WOB (Weight on bit)+ TQ (Torque) + Hookload + MWD (Measurements while drilling) + LWD (Logging while drilling) + WITSML (Well Site Information Transfer Standard Markup Language)
cosine similarity:  0.9954749943918912
------------
LRA data:  Disposal of surplus
LRA description:  Sales documention (description of equipment, certificates) + Sales order
 
Most similar BII data:  Disposal of surplus
Most similar BII description:  Sales documention (description of equipment, certificates) + Sales order

In [39]:
# For each data point in LRA finding all data in BII greater than the threshold. 
# Creating a [:STRONG_IM_LINK] relation between those nodes 

th = 0.995

for n in range(len(LRA_BII_cos)):
    for m in np.where(LRA_BII_cos[n] > th)[0]:
        print ("LRA ID: ", embeddings_LRA[n]['ID'])
        print ("LRA data: ", name_LRA[n])
        print ("LRA description: ", desc_LRA[n])
        print (" ")

        print ("BII ID: ", embeddings_BII[m]['ID'])
        print("Most similar BII data: ", name_BII[m])
        print("Most similar BII description: ", desc_BII[m])
        print ("cosine similarity: ", LRA_BII_cos[n][m])
        print ("------------")
        
        graph.query("MATCH (lra:LRA) WHERE ID(lra) = " + str(embeddings_LRA[n]['ID']) + " MATCH (bii:BII) WHERE ID(bii) = " + str( embeddings_BII[m]['ID']) + " MERGE (bii)-[:STRONG_IM_LINK]-(lra)")


LRA ID:  0
LRA data:  Real-time well data
LRA description:  Real-Time drilling and logging data - Incl.cement log data. SiteCom + ROP (Rate of Penetration) + WOB (Weight on bit)+ TQ (Torque) + Hookload + MWD (Measurements while drilling) + LWD (Logging while drilling) + WITSML (Well Site Information Transfer Standard Markup Language)
 
BII ID:  2650
Most similar BII data:  Real-Time log data
Most similar BII description:  Real-Time drilling and logging data - Incl.cement log data. SiteCom + ROP (Rate of Penetration) + WOB (Weight on bit)+ TQ (Torque) + Hookload + MWD (Measurements while drilling) + LWD (Logging while drilling) + WITSML (Well Site Information Transfer Standard Markup Language)
cosine similarity:  0.9954749943918912
------------
LRA ID:  324
LRA data:  Disposal of surplus
LRA description:  Sales documention (description of equipment, certificates) + Sales order
 
BII ID:  2506
Most similar BII data:  Disposal of surplus
Most similar BII description:  Sales documention (d

### LRA x LRA similarity

In [260]:
# Calculating a matrix with cosine similarity between LRA x LRA
LRA_LRA_cos = cosine_similarity(np.array(emb_LRA), np.array(emb_LRA))

In [265]:
# For each data point in LRA finding all data in LRA greater than the threshold. 
# Creating a [:STRONG_IM_LINK] relation between those nodes 

th = 0.995

for n in range(len(LRA_LRA_cos)):
    for m in np.where(LRA_LRA_cos[n] > th)[0]:
        if embeddings_LRA[n]['ID'] != embeddings_LRA[m]['ID']:
            # print ("LRA ID: ", embeddings_LRA[n]['ID'])
            # print ("LRA data: ", name_LRA[n])
            # print ("LRA description: ", desc_LRA[n])
            # print (" ")
    
            # print ("LRA ID: ", embeddings_LRA[m]['ID'])
            # print("Most similar LRA data: ", name_LRA[m])
            # print("Most similar LRA description: ", desc_LRA[m])
            # print ("cosine similarity: ", LRA_LRA_cos[n][m])
            # print ("------------")
            
            graph.query("MATCH (lra:LRA) WHERE ID(lra) = " + str(embeddings_LRA[n]['ID']) + " MATCH (lra2:LRA) WHERE ID(lra2) = " + str( embeddings_LRA[m]['ID']) + " MERGE (lra)-[:STRONG_IM_LINK]-(lra2)")

### BII x BII similarity

In [268]:
# Calculating a matrix with cosine similarity between LRA x LRA
BII_BII_cos = cosine_similarity(np.array(emb_BII), np.array(emb_BII))

In [269]:
# For each data point in BII finding all data in BII greater than the threshold. 
# Creating a [:STRONG_IM_LINK] relation between those nodes 

th = 0.995

for n in range(len(BII_BII_cos)):
    for m in np.where(BII_BII_cos[n] > th)[0]:
        if embeddings_BII[n]['ID'] != embeddings_BII[m]['ID']:
            print ("BII ID: ", embeddings_BII[n]['ID'])
            print ("BII data: ", name_BII[n])
            print ("BII description: ", desc_BII[n])
            print (" ")
    
            print ("BII ID: ", embeddings_BII[m]['ID'])
            print("Most similar BII data: ", name_BII[m])
            print("Most similar BII description: ", desc_BII[m])
            print ("cosine similarity: ", BII_BII_cos[n][m])
            print ("------------")
            
            graph.query("MATCH (bii:BII) WHERE ID(bii) = " + str(embeddings_BII[n]['ID']) + " MATCH (bii2:BII) WHERE ID(bii2) = " + str( embeddings_BII[m]['ID']) + " MERGE (bii)-[:STRONG_IM_LINK]-(bii2)")

BII ID:  2119
BII data:  Safety risk documentation
BII description:  Includes risk evaluation, assessment, decision and report. E.g. Hazid report
 
BII ID:  2477
Most similar BII data:  Notification of incident
Most similar BII description:  All correspondence related to notification of incident to relevant entities. Examples of typical documents are notification form, follow-up correspondence (e.g. documentation dispatch)
cosine similarity:  0.9953949952057396
------------
BII ID:  2155
BII data:  Geophysical reports (PETEC)
BII description:  Interpretation, processing or other geophysical reports
 
BII ID:  2169
Most similar BII data:  Geological reports (PETEC)
Most similar BII description:  Geological description, characterization and evaluation reports
cosine similarity:  0.9954115932184938
------------
BII ID:  2169
BII data:  Geological reports (PETEC)
BII description:  Geological description, characterization and evaluation reports
 
BII ID:  2155
Most similar BII data:  Geophy

### Clustering

In [295]:
#Join the lists from LRA and BII in the same arrays 
clust_emb = np.array(emb_LRA + emb_BII)
clust_name = np.array(name_LRA + name_BII)
clust_desc = np.array(desc_LRA + desc_BII)
clust_ID = np.array(ID_LRA + ID_BII)
clust_dataset = np.array(["LRA"] * len(emb_LRA) + ["BII"] * len(emb_BII))


In [296]:
# Clustering algorithm with minimun of 5 elemnets with maximuns distance of "alpha"
alpha = 0.008
clustering = DBSCAN(eps=alpha, min_samples=5, metric='cosine').fit(clust_emb)
#clustering = KMeans(n_clusters=500, random_state=0, n_init="auto").fit(clust_emb)

Reductions of dimensionality and visualization

In [297]:
#PCA_emb = PCA(n_components=2).fit_transform(clust_emb)

In [298]:
TSNE_emb = TSNE(n_components=2, learning_rate='auto', init='random', perplexity=3).fit_transform(clust_emb)

In [299]:
fig, ax = plt.subplots(figsize=(20, 20))#, layout='constrained')

plt.scatter(TSNE_emb.T[0], TSNE_emb.T[1], c=clustering.labels_, s=2)

for i in range(len(clust_emb)):
    if clustering.labels_[i] > -1:
        plt.text(x=TSNE_emb.T[0][i], 
                 y=TSNE_emb.T[1][i],
                 #s='(' + str(clustering.labels_[i]) + ') ' + str(i), fontdict=dict(size=1),)
                 s='(' + str(clustering.labels_[i]) + ') ' + ' - ' + clust_dataset[i] + ' - ' + clust_name[i], fontsize= 2)
        # if i > 1000:
        #     break
    
mpld3.display(fig)

In [304]:
clu = 11
clust_n_name = clust_name[clustering.labels_ == clu]
clust_n_desc = clust_desc[clustering.labels_ == clu]
clust_n_ID = clust_ID[clustering.labels_ == clu]
clust_n_dataset = clust_dataset[clustering.labels_ == clu]

for data in range(len(clust_n_name)):
    print(clust_n_dataset[data], " / (", clust_n_ID[data], ") ", clust_n_name[data], " / ", clust_n_name[data])

LRA  / ( 185 )  Well integrity  /  Well integrity
LRA  / ( 4243 )  Well integrity  /  Well integrity
LRA  / ( 4457 )  Well integrity  /  Well integrity
BII  / ( 2373 )  Certificates (Incl. M4 PM Certificate)  /  Certificates (Incl. M4 PM Certificate)
BII  / ( 2494 )  License/Lease - equity and administrative  /  License/Lease - equity and administrative


In [325]:
# Creating a "Data Product" node for each cluster and connect the data points

for clu in set(clustering.labels_):
    if clu != -1:
        
        graph.query("MERGE (n:DataProduct {number: " + str(clu) +  "})")
        print ("MERGE DataProduct number ", clu)
        #graph.query("MATCH (bii:BII) WHERE ID(bii) = " + str(embeddings_BII[n]['ID']) + " MATCH (bii2:BII) WHERE ID(bii2) = " + str( embeddings_BII[m]['ID']) + " MERGE (bii)-[:STRONG_IM_LINK]-(bii2)")

        clust_n_ID = clust_ID[clustering.labels_ == clu]
        clust_n_dataset = clust_dataset[clustering.labels_ == clu]

        for data in range(len(clust_n_ID)):

            graph.query("MATCH (n:DataProduct {number: " + str(clu)  + "}) MATCH (m:" + clust_n_dataset[data] + ") WHERE ID(m) = " + str(clust_n_ID[data]) + " MERGE (m)-[:PART_OF]-(n);")
            print("MERGE  (n:DataProduct {number: " + str(clu)  + "}) <-[:PART_OF]-  (m:" + clust_n_dataset[data] + " ID(" + str(clust_n_ID[data]) + ")) ")


MERGE DataProduct number  0
MERGE  (n:DataProduct {number: 0}) <-[:PART_OF]-  (m:LRA ID(15)) 
MERGE  (n:DataProduct {number: 0}) <-[:PART_OF]-  (m:LRA ID(28)) 
MERGE  (n:DataProduct {number: 0}) <-[:PART_OF]-  (m:LRA ID(29)) 
MERGE  (n:DataProduct {number: 0}) <-[:PART_OF]-  (m:LRA ID(36)) 
MERGE  (n:DataProduct {number: 0}) <-[:PART_OF]-  (m:LRA ID(37)) 
MERGE  (n:DataProduct {number: 0}) <-[:PART_OF]-  (m:LRA ID(52)) 
MERGE  (n:DataProduct {number: 0}) <-[:PART_OF]-  (m:LRA ID(61)) 
MERGE  (n:DataProduct {number: 0}) <-[:PART_OF]-  (m:LRA ID(76)) 
MERGE  (n:DataProduct {number: 0}) <-[:PART_OF]-  (m:LRA ID(77)) 
MERGE  (n:DataProduct {number: 0}) <-[:PART_OF]-  (m:LRA ID(84)) 
MERGE  (n:DataProduct {number: 0}) <-[:PART_OF]-  (m:LRA ID(100)) 
MERGE  (n:DataProduct {number: 0}) <-[:PART_OF]-  (m:LRA ID(101)) 
MERGE  (n:DataProduct {number: 0}) <-[:PART_OF]-  (m:LRA ID(171)) 
MERGE  (n:DataProduct {number: 0}) <-[:PART_OF]-  (m:LRA ID(271)) 
MERGE  (n:DataProduct {number: 0}) <-[:PART_